#Exercise for multiclass classification

#Check for GPU

#Imports

#Get dataset

#Walkthrough dataset folders


#Prepare datast=et folders

#Use Text dataset from directory

#Look Through Dataset

#Create val & test datasets

#Text vectorization

#Perf

#Create the model

#Evaluate the model

#Plot loss curves
